# <font color=darkblue> Machine Learning model deployment with Flask framework on Heroku</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

### 2. Load the dataset

In [2]:
df = pd.read_csv('car+data.csv')
df.head()

Car_Name  Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type  \
0     ritz  2014           3.35           5.59       27000    Petrol   
1      sx4  2013           4.75           9.54       43000    Diesel   
2     ciaz  2017           7.25           9.85        6900    Petrol   
3  wagon r  2011           2.85           4.15        5200    Petrol   
4    swift  2014           4.60           6.87       42450    Diesel   

  Seller_Type Transmission  Owner  
0      Dealer       Manual      0  
1      Dealer       Manual      0  
2      Dealer       Manual      0  
3      Dealer       Manual      0  
4      Dealer       Manual      0

### 3. Check the shape and basic information of the dataset.

In [3]:
df.shape

(301, 9)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


- We have around 300 instances and 9 features
- There are 4 categorical features and 5 numerical features.
- Also, We can observe that there are no missing values present in the dataset.

In [5]:
df.describe()

Year  Selling_Price  Present_Price     Kms_Driven       Owner
count   301.000000     301.000000     301.000000     301.000000  301.000000
mean   2013.627907       4.661296       7.628472   36947.205980    0.043189
std       2.891554       5.082812       8.644115   38886.883882    0.247915
min    2003.000000       0.100000       0.320000     500.000000    0.000000
25%    2012.000000       0.900000       1.200000   15000.000000    0.000000
50%    2014.000000       3.600000       6.400000   32000.000000    0.000000
75%    2016.000000       6.000000       9.900000   48767.000000    0.000000
max    2018.000000      35.000000      92.600000  500000.000000    3.000000

In [6]:
df.describe(include='O')

Car_Name Fuel_Type Seller_Type Transmission
count       301       301         301          301
unique       98         3           2            2
top        city    Petrol      Dealer       Manual
freq         26       239         195          261

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [7]:
len(df[df.duplicated()])

2

- We can observe that there are 2 duplicate records, lets drop them.

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
len(df[df.duplicated()])

0

### 5. Drop the columns which you think redundant for the analysis.

In [10]:
df=df.drop(columns=['Car_Name'],axis=1)

In [11]:
df.columns

Index(['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven', 'Fuel_Type',
       'Seller_Type', 'Transmission', 'Owner'],
      dtype='object')

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [12]:
df['age_of_the_car']=2023-df['Year']

In [13]:
df=df.drop(columns=['Year'],axis=1)

In [14]:
df.columns

Index(['Selling_Price', 'Present_Price', 'Kms_Driven', 'Fuel_Type',
       'Seller_Type', 'Transmission', 'Owner', 'age_of_the_car'],
      dtype='object')

In [15]:
df.head(20)

Selling_Price  Present_Price  Kms_Driven Fuel_Type Seller_Type  \
0            3.35           5.59       27000    Petrol      Dealer   
1            4.75           9.54       43000    Diesel      Dealer   
2            7.25           9.85        6900    Petrol      Dealer   
3            2.85           4.15        5200    Petrol      Dealer   
4            4.60           6.87       42450    Diesel      Dealer   
5            9.25           9.83        2071    Diesel      Dealer   
6            6.75           8.12       18796    Petrol      Dealer   
7            6.50           8.61       33429    Diesel      Dealer   
8            8.75           8.89       20273    Diesel      Dealer   
9            7.45           8.92       42367    Diesel      Dealer   
10           2.85           3.60        2135    Petrol      Dealer   
11           6.85          10.38       51000    Diesel      Dealer   
12           7.50           9.94       15000    Petrol      Dealer   
13           6.10           7.71       26000    Petrol      Dealer   
14           2.25           7.21       77427    Petrol      Dealer   
15           7.75          10.79       43000    Diesel      Dealer   
16           7.25          10.79       41678    Diesel      Dealer   
18           3.25           5.09       35500       CNG      Dealer   
19           2.65           7.98       41442    Petrol      Dealer   
20           2.85           3.95       25000    Petrol      Dealer   

   Transmission  Owner  age_of_the_car  
0        Manual      0               9  
1        Manual      0              10  
2        Manual      0               6  
3        Manual      0              12  
4        Manual      0               9  
5        Manual      0               5  
6        Manual      0               8  
7        Manual      0               8  
8        Manual      0               7  
9        Manual      0               8  
10       Manual      0               6  
11       Manual      0               8  
12    Automatic      0               8  
13       Manual      0               8  
14       Manual      0              14  
15       Manual      0               7  
16       Manual      0               8  
18       Manual      0               8  
19       Manual      0              13  
20       Manual      0               7

### 7. Encode the categorical columns

In [16]:
df_cat = df.select_dtypes(include='object')
le = LabelEncoder()
for col in df_cat:
    df[col] = le.fit_transform(df[col])

In [17]:
df.head(20)

Selling_Price  Present_Price  Kms_Driven  Fuel_Type  Seller_Type  \
0            3.35           5.59       27000          2            0   
1            4.75           9.54       43000          1            0   
2            7.25           9.85        6900          2            0   
3            2.85           4.15        5200          2            0   
4            4.60           6.87       42450          1            0   
5            9.25           9.83        2071          1            0   
6            6.75           8.12       18796          2            0   
7            6.50           8.61       33429          1            0   
8            8.75           8.89       20273          1            0   
9            7.45           8.92       42367          1            0   
10           2.85           3.60        2135          2            0   
11           6.85          10.38       51000          1            0   
12           7.50           9.94       15000          2            0   
13           6.10           7.71       26000          2            0   
14           2.25           7.21       77427          2            0   
15           7.75          10.79       43000          1            0   
16           7.25          10.79       41678          1            0   
18           3.25           5.09       35500          0            0   
19           2.65           7.98       41442          2            0   
20           2.85           3.95       25000          2            0   

    Transmission  Owner  age_of_the_car  
0              1      0               9  
1              1      0              10  
2              1      0               6  
3              1      0              12  
4              1      0               9  
5              1      0               5  
6              1      0               8  
7              1      0               8  
8              1      0               7  
9              1      0               8  
10             1      0               6  
11             1      0               8  
12             0      0               8  
13             1      0               8  
14             1      0              14  
15             1      0               7  
16             1      0               8  
18             1      0               8  
19             1      0              13  
20             1      0               7

- We observe that the Encoded dataset with original dataset, labels of the converted categorical column values are
- Fuel_Type labels
    - CNG   : 0
    - diesel: 1
    - petrol: 2
- Seller_Type labels
    - Dealer     : 0
    - Individual : 1
- Transmission
    - Manual    : 0
    - Automatic : 1

### 8. Separate the target and independent features.

In [18]:
X= df.drop('Selling_Price',axis=1)
y= df['Selling_Price']

### 9. Split the data into train and test.

In [19]:
## train_test_split() is used to divide dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=0)
print('Shape of X_train:',X_train.shape)
print('Shape of X_test:',X_test.shape)
print('Shape of y_train:',y_train.shape)
print('Shape of y_test:',y_test.shape)

Shape of X_train: (209, 7)
Shape of X_test: (90, 7)
Shape of y_train: (209,)
Shape of y_test: (90,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [20]:
rf = RandomForestRegressor()
rf.fit(X_train,y_train)

RandomForestRegressor()

In [21]:
y_train_pred = rf.predict(X_train) 
y_test_pred = rf.predict(X_test)

r2_train = r2_score(y_train,y_train_pred)
r2_test = r2_score(y_test,y_test_pred)

print('r2-score train:', r2_train)
print('r2-score test:', r2_test)

r2-score train: 0.9746804667183184
r2-score test: 0.9058382272287525


### 11. Create a pickle file with an extension as .pkl

In [22]:
import pickle
#Saving model to disk
pickle.dump(rf, open('model.pkl', 'wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>

<body>

    <div class="hero-image">
        <div class="hero-text">

            <h1 style="font-size:50px">Used Car Price Predictor</h1>
            <br><br>
            <h3>{{ prediction_text }}<h3>
        </div>
    </div>

    <style>
        body,
        html {
            height: 100%;
            margin: 0;
            font-family: Arial, Helvetica, sans-serif;
        }

        .hero-image {
            background-image: linear-gradient(rgba(0, 0, 0, 0.5), rgba(0, 0, 0, 0.5)), url('/static/image.jpg');
            height: 50%;
            background-position: bottom;
            background-repeat: no-repeat;
            background-size: cover;
            position: relative;
        }

        .hero-text {
            text-align: center;
            position: absolute;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
            color: white;
        }
    </style>


    <div style="color:	rgb(0, 0, 0)">
        <form action="{{ url_for('predict')}}" method="post">
            <h2>Enter Car Details: </h2>
            <h3>Age of the car(In years)</h3>
            <input id="first" name="Age_of_the_car" type="number ">
            <h3>Present Showroom Price(In lakhs)</h3><br><input id="second" name="Present_Price" required="required">
            <h3>Kilometers Driven</h3><input id="third" name="Kms_Driven" required="required">
            <h3>Owner Type (0/1/3)</h3><br><input id="fourth" name="Owner" required="required">
            <h3>Fuel type</h3><br><select name="Fuel_Type" id="fuel" required="required">
                <option value="0">Petrol</option>
                <option value="1">Diesel</option>
                <option value="2">CNG</option>
            </select>
            <h3>Seller Type</h3><br><select name="Seller_Type" id="resea" required="required">
                <option value="0">Dealer</option>
                <option value="1">Individual</option>
            </select>
            <h3>Transmission type</h3><br><select name="Transmission" id="research" required="required">
                <option value="0">Manual Car</option>
                <option value="1">Automatic Car</option>
            </select>
            <br><br><button id="sub" type="submit ">Predict Selling Price</button>
            <br>


        </form>

    </div>

    <style>
        body {
            background-color: 101, 10, 20;
            text-align: center;
            padding: 0px;
            font-family: Helvetica;
        }

        #research {
            font-size: 18px;
            width: 200px;
            height: 23px;
            top: 23px;
        }

        #box {
            border-radius: 60px;
            border-color: 45px;
            border-style: solid;
            text-align: center;
            background-color: white;
            font-size: medium;
            position: absolute;
            width: 700px;
            bottom: 9%;
            height: 850px;
            right: 30%;
            padding: 0px;
            margin: 0px;
            font-size: 14px;
        }

        #fuel {
            width: 83px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 20px;
        }

        #fuel:hover {
            background-color: white;
        }

        #research {
            width: 150px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #research:hover {
            background-color: white;
        }

        #resea {
            width: 99px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #resea:hover {
            background-color: white;
        }

        #sub {
            background-color: Green;
            font-family: 'Helvetica' monospace;
            font-weight: bold;
            width: 180px;
            height: 60px;
            text-align: center;
            border-radius: 20px;
            font-size: 18px;
            color: white;
        }

        #sub:hover {
            background-color: white;
        }

        #first {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #second {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #third {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #fourth {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
    </style>
</body>

</html>

### b) Create app.py file and write the predict function

In [ ]:
from flask import Flask, render_template, request, jsonify
import pickle
import numpy as np
import sklearn


app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))


@app.route('/')
def home():
    return render_template('index.html')


@app.route("/predict", methods=['POST'])
def predict():
    if request.method == 'POST':
        Present_Price=float(request.form['Present_Price'])
        Kms_Driven=int(request.form['Kms_Driven'])
        Owner=int(request.form['Owner'])
        Fuel_Type=request.form['Fuel_Type']
        Age_of_the_car=request.form['Age_of_the_car']
        Seller_Type=request.form['Seller_Type']
        Transmission=request.form['Transmission']

        prediction=model.predict([[Present_Price,Kms_Driven,Owner,Age_of_the_car,Fuel_Type,Seller_Type,Transmission]])
        output=round(prediction[0],2)
        return render_template('index.html', prediction_text="You can sell your car at {} lakhs".format(output))

if __name__=="__main__":
    app.run(debug=True)

### 13. Deploy your app on Railway.app . (write commands for deployment)

1. create procfile, runtime, requirements files to the root folder
2. sign up with railway.app website, Github
3. click New project
4. go to the Github Repo
5. select the repo or configure the github to show the repository
6. then initialization done and the web link will be genereated after deployment.

### 14. Paste the URL of the Railway.app application below, and while submitting the solution submit this notebook along with the source code.

In [ ]:
https://web-production-5276.up.railway.app/

### Happy Learning :)